In [175]:
def typeCheck(a, b):
    if not isinstance(a, type(b)) and not isinstance(b, type(a)):
        raise Exception('Incompatible types')

# Forward

class BaseRoutine:
    pass

# Concepts

class Concept:
    def __init__(self, name: str):
        self.name = name
        self.items = {}
    
    def __call__(self):
        return self
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
basesToDerived = {}
derivedToBases = {}

class ConceptList(list):
    name: str
    
    def __init__(self, name: str):
        super().__init__()
        self.name = name
        
    def __call__(self):
        return self
        
    def __repr__(self):
        return f'{self.name}: {super().__repr__()}'

# Primitives

class PList(list):
    def __init__(self, obj: Concept):
        super(PList, self).__init__()
        self.type = type(obj)

def plist(obj: Concept) -> list:
    return PList(obj)

def count(plist: list) -> int:
    return len(plist)

def pop(plist: list) -> Concept:
    return plist.pop()

def add(plist: list, elt: Concept) -> None:
    plist.add(elt)
    
def remove(plist: list, elt: Concept) -> None:
    del plist[plist.index(elt)]
    
def pfilter(plist: list, fn: BaseRoutine, arg: Concept) -> list:
    return [elt for elt in plist if eq(fn(elt), arg)]
    
def transform(plist: list, fn: BaseRoutine) -> list:
    return [fn(elt) for elt in plist]

def pand(*args: bool) -> bool:
    return all(arg for arg in args)

def por(*args: bool) -> bool:
    return any(arg for arg in args)

def pnot(arg: bool) -> bool:
    return not arg

def has(container: Concept | list, field: str | Concept) -> bool:
    try:
        return field in container # PList, ObjectList
    except:
        return str(field) in container.items # Object

def isa(derived: Concept, base: Concept) -> bool:
    return derived in baseToDerived[base]

def get(obj: Concept, field: str | Concept) -> Concept:
    return obj.items[str(field)]

def index(plist: list, elt: Concept) -> int:
    return plist.index(elt)

def gt(a: int, b: int) -> bool:
    return a > b

def eq(a: Concept | int, b: Concept | int) -> bool:
    typeCheck(a, b)
    if type(a) == bool or type(b) == bool:
        return a == b and type(a) == type(b)
    else:
        return a == b
    
def lt(a: int, b: int) -> bool:
    return a < b

primitives = [plist, count, pop, add, remove, pfilter, transform, pand, por, pnot, has, isa, get, index, gt, eq, lt]

print([fn.__name__ for fn in primitives])

['plist', 'count', 'pop', 'add', 'remove', 'pfilter', 'transform', 'pand', 'por', 'pnot', 'has', 'isa', 'get', 'index', 'gt', 'eq', 'lt']


In [176]:
concepts = []

def getConcept(name):
    for c in concepts:
        if c.name == name:
            return c

def addToDict(dct, key, val):
    if key not in dct:
        dct[key] = []
    if val in dct[key]:
        raise 'Value already in dict[key]'
    dct[key].append(val)
        
def inheritFrom(derived, base):
    addToDict(derivedToBases, derived, base)
    addToDict(basesToDerived, base, derived)
        
concepts.append(Concept('Suit'))
concepts.append(Concept('Rank'))
suits = ['Hearts', 'Diamonds', 'Spades', 'Clubs']
ranks = ['6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
concepts.append(ConceptList('Ranks'))
for s in suits:
    s = Concept(s)
    concepts.append(s)
    inheritFrom(s, getConcept('Suit'))
for r in ranks:
    r = Concept(r)
    concepts.append(r)
    inheritFrom(r, getConcept('Rank'))
    getConcept('Ranks').append(r)
print(concepts)


[Suit, Rank, Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace], Hearts, Diamonds, Spades, Clubs, 6, 7, 8, 9, 10, Jack, Queen, King, Ace]


In [177]:
for k,v in basesToDerived.items():
    print((k,v))

(Suit, [Hearts, Diamonds, Spades, Clubs])
(Rank, [6, 7, 8, 9, 10, Jack, Queen, King, Ace])


In [212]:
from inspect import signature
from anytree import Node, NodeMixin, RenderTree, PreOrderIter
from itertools import chain, combinations, product

class Arg():
    def __init__(self, idx, arg):
        self.idx = idx
        self.arg = arg
    
    def __call__(self):
        return self.arg
    
    def __repr__(self):
        return self.arg.__repr__()

def getArg(arg):
    if type(arg) == Arg:
        return arg.arg
    elif type(arg) == Routine:
        if arg.result is None: # Transform primitive
            return arg
        else:
            return arg.result
    else:
        return arg
    
def allowTypePrimitive(obj, typ):
    if type(obj) == bool:
        return typ == bool
    else:
        return isinstance(obj, typ)
    
def allowTypeComposite(obj, typ):
    # int, bool, etc.
    if type(obj) == typ:
        return True
    # Rank, Card, Game, etc.
    if typ in basesToDerived[obj]:
        return True
    
def powerset(iterable):
    '''powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    '''
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

class Routine(BaseRoutine, NodeMixin):
    def __init__(self, func, name=None, parent=None, children=None, result=None, params=None, args=None):
        self.func = func
        self.sig = signature(self.func)
        self.name = name if name is not None else func.__name__
        self.parent = parent
        if children is not None:
            self.children = children
        self.result = result
        self.params = params
        self.n = len(self.sig.parameters) if self.func not in [por, pand] else -1
        self.args = args
        self.xargs = None
        self.memory = None
        
    def calls(self):
        '''invoke __call__ for any new arg combinations'''
        def helper(arg):
            return arg if type(arg) != Routine else None
        if self.memory is None:
            raise Exception("Haven't remembered")
        if self.n == -1:
            before = list(powerset(self.xargs[:self.memory]))
            after = list(powerset(self.xargs[self.memory:]))
            for item in product(before, after):
                if len(item) != 0:
                    copy = self.copy()
                    copy.args = [helper(arg) for arg in item]
                    copy.children = [arg.copy() for arg in item if type(arg) == Routine]
                    try:
                        copy()
                        yield copy
                    except Exception as ex:
                        pass
#                         print(ex)
            self.memory = len(self.xargs)
        else:
            args = [args[limit:] for args, limit in zip(self.xargs, self.memory)]
            for item in product(*args):
                print(item)
                copy = self.copy()
                copy.args = [helper(arg) for arg in item]
                copy.children = [arg.copy() for arg in item if type(arg) == Routine]
                try:
                    copy()
                    yield copy
                except Exception as ex:
                    if copy.func.__name__ == 'gt':
                        print(type(copy.args[0]))
                        print(copy.args[0].result)
                        print(copy.args[0].args)
                    print(copy.args[0])
                    print(copy.args[1])
                    print(type(copy.args[0]))
                    print(type(copy.args[1]))
                    print(ex)
            self.memory = [len(args) for args in self.xargs]
    
    def copy(self):
        return Routine(self.func, self.name, None, 
                       [child.copy() for child in self.children], self.result, self.params, self.args)
    
    def forget(self):
        self.xargs = None
        self.memory = None
    
    def remember(self):
        if self.n > -1:
            self.xargs = [[] for _ in range(self.n)]
            self.memory = self.n*[0]
        else:
            self.xargs = []
            self.memory = 0
    
    def slot(self, arg):
        '''Find compatible arg slots and place inside'''
        # And, or
        if self.n == -1:
            if type(getArg(arg)) == bool:
                if arg not in self.args:
                    self.args.append(arg)
        # Primitive
        elif self.params is None:
            for i,(hint,typ) in enumerate(self.func.__annotations__.items()):
                if hint == 'return':
                    continue
                if allowTypePrimitive(getArg(arg), typ):
                    if arg not in self.xargs[i]:
                        self.xargs[i].append(arg)
                    else:
                        print(arg)
        # Composite
        else:
            for i,hint in enumerate(self.params):
                if allowTypeComposite(getArg(arg), hint):
                    if arg not in self.xargs[i]:
                        self.xargs[i].append(arg)
    
    def __call__(self, *args):
        if len(args) == 0:
            args = self.args
            for child in self.children:
                args[args.index(None)] = child
            args = [arg() for arg in args]
        self.result = self.func(*args)
        return self.result
    
    def __eq__(self, other):
        return (type(other) == Routine and self.func == other.func 
                and self.params == other.params and self.result == other.result)
    
    def __repr__(self):
        return f'{self.name} {self.n} {self.result}'
    
print('Complete')

Complete


In [214]:
def makeArgs(*args):
    return [Arg(i,arg) for i,arg in enumerate(args)]

def stricteq(a, b):
    if type(a) == bool or type(b) == bool:
        return a == b and type(a) == type(b)
    else:
        return a == b

class AI:
    def __init__(self, concepts, primitives):
        self.concepts = concepts
        self.routines = [Routine(fn) for fn in primitives]
        
    def solve(self, name, conceptNames, routineNames, args, target):
        self.cons = [c for c in self.concepts if c.name in conceptNames]
        self.routes = [rt for rt in self.routines if rt.name in routineNames]
        self.leaves = []
        self.matches = []
        for rt in self.routes:
            rt.forget()
            rt.remember()
            for arg in self.cons+args+self.routes:
                rt.slot(arg)
        for i in range(5):
            for rt in self.routes:
                for arg in self.leaves:
                    rt.slot(arg)
                for leaf in rt.calls():
                    print(f'res: {leaf.result}')
                    if stricteq(leaf.result, target):
                        self.matches.append(leaf)
                    self.leaves.append(leaf)
#         for rt in self.routes:
#             rt.forget()
        return len(self.matches) > 0
    
    def __str__(self):
        return f'Concepts: {self.concepts}\n Routines: {self.routines}'
    
ai = AI(concepts, primitives)
print(str(ai))

routes = ['index', 'gt']
cons = ['Ranks', 'Rank']
rankA = getConcept('Ranks')[2]
rankB = getConcept('Ranks')[0]
ai.solve('rankgt', cons, routes, makeArgs(rankA, rankB), True)

Concepts: [Suit, Rank, Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace], Hearts, Diamonds, Spades, Clubs, 6, 7, 8, 9, 10, Jack, Queen, King, Ace]
 Routines: [plist 1 None, count 1 None, pop 1 None, add 2 None, remove 2 None, pfilter 3 None, transform 2 None, pand -1 None, por -1 None, pnot 1 None, has 2 None, isa 2 None, get 2 None, index 2 None, gt 2 None, eq 2 None, lt 2 None]
(Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace], Rank)
Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace]
Rank
<class '__main__.ConceptList'>
<class '__main__.Concept'>
Rank is not in list
(Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace], 8)
res: 2
(Ranks: [6, 7, 8, 9, 10, Jack, Queen, King, Ace], 6)
res: 0
(index 2 2, index 2 2)
res: False
(index 2 2, index 2 0)
res: True
(index 2 0, index 2 2)
res: False
(index 2 0, index 2 0)
res: False
index 2 2
index 2 2
index 2 0
index 2 0
index 2 2
index 2 2
index 2 0
index 2 0
index 2 2
index 2 2
index 2 0
index 2 0
index 2 2
index 2 2
index 2 0
index 2 0


True

In [156]:
ai.routes[2].xargs

[[], []]

In [157]:
ai.routes

[get 2 None, index 2 None, gt 2 None]

In [116]:
isinstance(getConcept('Rank'), ai.routes[0].func.__annotations__['obj'])

True

In [135]:
isinstance(getArg(getConcept('Ranks')), ai.routes[1].func.__annotations__['elt'])

False

In [138]:
isinstance(getConcept('Ranks'), Concept)

True

In [79]:
ai.routes[0].func.__annotations__['obj']

__main__.Object

In [90]:
isinstance(concepts[1], Concept)

False

In [164]:
get(getConcept('Rank'), getConcept('6'))

KeyError: '6'

In [205]:
ai.leaves

[index 2 None, index 2 None, gt 2 None]